In [140]:
import pandas as pd
import numpy as np

In [142]:
df = pd.read_csv("walmart_train.csv")

In [144]:
df['Date'] = pd.to_datetime(df['Date'])

In [146]:
df = df[df['Weekly_Sales'] > 0]
df = df.dropna()

In [148]:
topStore = (
    df.groupby('Store')['Weekly_Sales']
      .sum()
      .idxmax()
)

In [150]:
df_store = df[df['Store'] == topStore] # 20 nolu magaza

In [152]:
top100Depts = (
    df_store.groupby('Dept')['Weekly_Sales']
            .sum()
            .sort_values(ascending=False)
            .head(100)
            .index
)

In [154]:
df_final = df_store[df_store['Dept'].isin(top100Depts)]

In [156]:
df_final = df_final.drop(columns=['Store','IsHoliday'])

In [158]:
df_final = df_final.sort_values(by='Date')

In [160]:
df_final.head()

,Dept,Date,Weekly_Sales
187335,1,2010-02-05,46021.21
195189,74,2010-02-05,26726.10
188193,7,2010-02-05,33042.19
195046,72,2010-02-05,173075.66
194903,71,2010-02-05,20512.14


In [162]:
df_grouped = df_final.groupby('Dept')['Weekly_Sales'].mean().reset_index()

In [164]:
df_grouped.rename(columns={'Weekly_Sales': 'Value'}, inplace=True)

In [166]:
df_grouped['Value'] = df_grouped['Value'].astype(int)

In [168]:
np.random.seed(42) 
random_margins = np.random.uniform(0.05, 0.40, size=len(df_grouped))

In [171]:
df_grouped['Cost'] = (df_grouped['Value'] * (1 - random_margins)).astype(int)

# Cost 0 veya negatif olmasın diye kontrol (Güvenlik)
df_grouped['Cost'] = df_grouped['Cost'].apply(lambda x: x if x > 0 else 1)

# 5. Sütun sırasını Java'nın seveceği hale getirelim: Dept, Cost, Value
df_final = df_grouped[['Dept', 'Cost', 'Value']]

In [173]:
df_final.to_csv("cleanedWalmartTopStore.csv", index=False)

In [195]:
df_final

,Dept,Cost,Value
0,1,33202,40545
1,2,48300,78251
2,3,10746,15490
3,4,38101,51456
4,5,37290,41647
...,...,...,...
73,95,100095,150613
74,96,14,21
75,97,17952,25836
76,98,13114,19284


In [191]:
tum_deptler = set(range(1, 100))
mevcut_deptler = set(df['Dept'].unique())

# 3. Aradaki farkı bul (Olmayanlar)
olmayan_deptler = tum_deptler - mevcut_deptler

# 4. Sonucu sıralı bir şekilde yazdır
print(f"Toplam Olmayan Departman Sayısı: {len(olmayan_deptler)}")
print("Olmayan Departman Numaraları:")
print(sorted(list(olmayan_deptler)))

Toplam Olmayan Departman Sayısı: 18
Olmayan Departman Numaraları:
[15, 53, 57, 61, 62, 63, 64, 66, 68, 69, 70, 73, 75, 76, 84, 86, 88, 89]
